## Imports

In [ ]:
# Basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# MlFlow
import mlflow
import mlflow.sklearn

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV

# Model Abstraction
import model_functions as ModelFunctions
from model_functions import PARAM_GRID_RF, PARAM_GRID_GB, PARAM_GRID_KNN

# Warnings
import warnings
warnings.filterwarnings("ignore")

# Path
_ = load_dotenv()
FULL_PATH = os.environ.get('FULL_PATH')
PATH = os.path.join(FULL_PATH, 'datasets')

# Case: Predição de churn
___

Vamos trabalhar com a análise [desta base de dados](https://www.kaggle.com/datasets/shilongzhuang/telecom-customer-churn-by-maven-analytics?resource=download).

O churn é um dos principais problemas enfrentados por empresas dos mais diversos segmentos. É de muito interesse das empresas amenizar esse tipo de problema, já que o churn implica em perdas financeiras que podem ser bastante significativas.

Pensando nisso, você recebeu a tarefa de estudar o comportamento histórico de uma base de alguns usuários de uma empresa no setor de telecomunicações. Sua tarefa é identificar quais podem ser os principais fatores associados ao churn dos clientes, e desenhar estratégias para tentar antever o problema e tomar as devidas ações para minimizá-lo.

Por fim, você também deve produzir uma apresentação de Negócios com os principais resultados e recomendações.

Algumas perguntas para guiar a análise:
- Qual o perfil de cliente mais propenso ao churn?
- Quais os principais fatores envolvidos no churn dos clientes?
- Como se antecipar ao problema?
- Como conectar a análise a indicadores de negócio?

## Datasets

In [138]:
churn_path = os.path.join(PATH, 'telecom_customer_churn.csv')
df_churn = pd.read_csv(churn_path)

In [139]:
df_churn.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,9,NaN,Yes,42.39,No,Yes,Cable,16.0,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10.0,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30.0,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4.0,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11.0,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [140]:
churn_columns = df_churn.columns.tolist()
churn_columns

['Customer ID',
 'Gender',
 'Age',
 'Married',
 'Number of Dependents',
 'City',
 'Zip Code',
 'Latitude',
 'Longitude',
 'Number of Referrals',
 'Tenure in Months',
 'Offer',
 'Phone Service',
 'Avg Monthly Long Distance Charges',
 'Multiple Lines',
 'Internet Service',
 'Internet Type',
 'Avg Monthly GB Download',
 'Online Security',
 'Online Backup',
 'Device Protection Plan',
 'Premium Tech Support',
 'Streaming TV',
 'Streaming Movies',
 'Streaming Music',
 'Unlimited Data',
 'Contract',
 'Paperless Billing',
 'Payment Method',
 'Monthly Charge',
 'Total Charges',
 'Total Refunds',
 'Total Extra Data Charges',
 'Total Long Distance Charges',
 'Total Revenue',
 'Customer Status',
 'Churn Category',
 'Churn Reason']

In [141]:
df_churn.shape

(7043, 38)

## Analise


## Tratando Dataframe

In [142]:
df_churn_filter = df_churn.copy()
df_churn_filter = df_churn_filter.drop(columns = [
    'City',
    'Zip Code',
    'Latitude',
    'Longitude',
    'Churn Category',
    'Churn Reason'
    ]
)

### Variaveis Categóricas para Numéricas

In [143]:
df_churn_filter.head()

,Customer ID,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
0,0002-ORFBO,Female,37,Yes,0,2,9,NaN,Yes,42.39,No,Yes,Cable,16.0,No,Yes,No,Yes,Yes,No,No,Yes,One Year,Yes,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed
1,0003-MKNFE,Male,46,No,0,0,9,NaN,Yes,10.69,Yes,Yes,Cable,10.0,No,No,No,No,No,Yes,Yes,No,Month-to-Month,No,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed
2,0004-TLHLJ,Male,50,No,0,0,4,Offer E,Yes,33.65,No,Yes,Fiber Optic,30.0,No,No,Yes,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned
3,0011-IGKFF,Male,78,Yes,0,1,13,Offer D,Yes,27.82,No,Yes,Fiber Optic,4.0,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned
4,0013-EXCHZ,Female,75,Yes,0,3,3,NaN,Yes,7.38,No,Yes,Fiber Optic,11.0,No,No,No,Yes,Yes,No,No,Yes,Month-to-Month,Yes,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned


In [144]:
def yes_no_num(string: str) -> int:
    if string == 'Yes':
        return 1
    return 0

def gender_num(string: str) -> int:
    if string == 'Male':
        return 0
    return 1

def offer_num(string: str) -> int:
    if string == 'Offer A':
        return 1
    elif string == 'Offer B':
        return 2
    elif string == 'Offer C':
        return 3
    elif string == 'Offer D':
        return 4
    elif string == 'Offer E':
        return 5
    return 0

def internet_type_num(string: str) -> int:
    if string == 'Fiber Optic':
        return 1
    elif string == 'DSL':
        return 2
    elif string == 'Cable':
        return 3
    return 0

def contract_num(string: str) -> int:
    if string == 'Month-to-Month':
        return 1
    elif string == 'Two Year':
        return 2
    elif string == 'One Year':
        return 3
    return 0

def payment_method_num(string: str) -> int:
    if string == 'Bank Withdrawal':
        return 1
    elif string == 'Credit Card':
        return 2
    elif string == 'Mailed Check':
        return 3
    return 0

def customer_status_num(string: str) -> int:
    if string == 'Stayed':
        return 1
    elif string == 'Joined':
        return 2    
    return 0

def age_bracket(age: int) -> int:
    if age >= 78:
        return 5
    elif age >= 59:
        return 4
    elif age >= 43:
        return 3
    elif age >= 27:
        return 2
    else:
        return 1

In [145]:
yes_no_columns = ['Married', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 
                  'Device Protection Plan', 'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music',
                  'Unlimited Data', 'Paperless Billing']

In [146]:
for column in yes_no_columns:
    df_churn_filter[column] = df_churn_filter[column].apply(yes_no_num)
df_churn_filter['Gender'] = df_churn_filter['Gender'].apply(gender_num)
df_churn_filter['Age'] = df_churn_filter['Age'].apply(age_bracket)
df_churn_filter['Offer'] = df_churn_filter['Offer'].apply(offer_num)
df_churn_filter['Internet Type'] = df_churn_filter['Internet Type'].apply(internet_type_num)
df_churn_filter['Contract'] = df_churn_filter['Contract'].apply(contract_num)
df_churn_filter['Payment Method'] = df_churn_filter['Payment Method'].apply(payment_method_num)
df_churn_filter['Customer Status'] = df_churn_filter['Customer Status'].apply(customer_status_num)

In [147]:
df_churn_filter.head()

,Customer ID,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
0,0002-ORFBO,1,2,1,0,2,9,0,1,42.39,0,1,3,16.0,0,1,0,1,1,0,0,1,3,1,2,65.6,593.30,0.00,0,381.51,974.81,1
1,0003-MKNFE,0,3,0,0,0,9,0,1,10.69,1,1,3,10.0,0,0,0,0,0,1,1,0,1,0,2,-4.0,542.40,38.33,10,96.21,610.28,1
2,0004-TLHLJ,0,3,0,0,0,4,5,1,33.65,0,1,1,30.0,0,0,1,0,0,0,0,1,1,1,1,73.9,280.85,0.00,0,134.60,415.45,0
3,0011-IGKFF,0,5,1,0,1,13,4,1,27.82,0,1,1,4.0,0,1,1,0,1,1,0,1,1,1,1,98.0,1237.85,0.00,0,361.66,1599.51,0
4,0013-EXCHZ,1,4,1,0,3,3,0,1,7.38,0,1,1,11.0,0,0,0,1,1,0,0,1,1,1,2,83.9,267.40,0.00,0,22.14,289.54,0


### Preenchendo NaN em variáveis numéricas

In [148]:
numeric_colums = ['Avg Monthly Long Distance Charges', 'Avg Monthly GB Download', 'Monthly Charge', 
                  'Total Charges', 'Total Refunds', 'Total Long Distance Charges', 'Total Revenue']

In [149]:
for column in numeric_colums:
    df_churn_filter[column] = df_churn_filter[column].fillna(df_churn_filter[column].mean())

In [150]:
df_churn_filter

,Customer ID,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
0,0002-ORFBO,1,2,1,0,2,9,0,1,42.390000,0,1,3,16.0,0,1,0,1,1,0,0,1,3,1,2,65.60,593.30,0.00,0,381.51,974.81,1
1,0003-MKNFE,0,3,0,0,0,9,0,1,10.690000,1,1,3,10.0,0,0,0,0,0,1,1,0,1,0,2,-4.00,542.40,38.33,10,96.21,610.28,1
2,0004-TLHLJ,0,3,0,0,0,4,5,1,33.650000,0,1,1,30.0,0,0,1,0,0,0,0,1,1,1,1,73.90,280.85,0.00,0,134.60,415.45,0
3,0011-IGKFF,0,5,1,0,1,13,4,1,27.820000,0,1,1,4.0,0,1,1,0,1,1,0,1,1,1,1,98.00,1237.85,0.00,0,361.66,1599.51,0
4,0013-EXCHZ,1,4,1,0,3,3,0,1,7.380000,0,1,1,11.0,0,0,0,1,1,0,0,1,1,1,2,83.90,267.40,0.00,0,22.14,289.54,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,1,1,0,0,0,13,4,1,46.680000,0,1,2,59.0,1,0,0,1,0,0,1,1,3,0,2,55.15,742.90,0.00,0,606.84,1349.74,1
7039,9992-RRAMN,0,2,1,0,1,22,4,1,16.200000,1,1,1,17.0,0,0,0,0,0,1,1,1,1,1,1,85.10,1873.70,0.00,0,356.40,2230.10,0
7040,9992-UJOEL,0,1,0,0,0,2,5,1,18.620000,0,1,2,51.0,0,1,0,0,0,0,0,1,1,1,2,50.30,92.75,0.00,0,37.24,129.99,2
7041,9993-LHIEB,0,1,1,0,5,67,1,1,2.120000,0,1,3,58.0,1,0,1,1,0,1,1,1,2,0,2,67.85,4627.65,0.00,0,142.04,4769.69,1


### Salvando Dataset do modelo

In [151]:
filtered_path = os.path.join(PATH, 'churn_model_dataset.csv')
df_churn_filter.to_csv(filtered_path, index=False)

## Dataframe Filtrado


In [194]:
df_churn_model = pd.read_csv(filtered_path)

In [195]:
df_churn_model_churned = df_churn_model[df_churn_model['Customer Status']==0].copy()
df_churn_model_churned.head()

,Customer ID,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
2,0004-TLHLJ,0,3,0,0,0,4,5,1,33.650000,0,1,1,30.0,0,0,1,0,0,0,0,1,1,1,1,73.9,280.85,0.0,0,134.60,415.45,0
3,0011-IGKFF,0,5,1,0,1,13,4,1,27.820000,0,1,1,4.0,0,1,1,0,1,1,0,1,1,1,1,98.0,1237.85,0.0,0,361.66,1599.51,0
4,0013-EXCHZ,1,4,1,0,3,3,0,1,7.380000,0,1,1,11.0,0,0,0,1,1,0,0,1,1,1,2,83.9,267.40,0.0,0,22.14,289.54,0
18,0022-TCJCI,0,5,0,0,0,45,0,1,10.670000,0,1,2,17.0,1,0,1,0,0,1,0,1,3,0,2,62.7,2791.50,0.0,0,480.15,3271.65,0
19,0023-HGHWL,0,4,0,0,0,1,0,0,25.420517,0,1,3,9.0,0,0,0,0,0,0,0,1,1,1,1,25.1,25.10,0.0,0,0.00,25.10,0


In [196]:
df_churn_model_churned.shape

(1869, 32)

In [197]:
df_churn_model_stayed = df_churn_model[df_churn_model['Customer Status']>0].copy()
df_churn_model_stayed.head()

,Customer ID,Gender,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
0,0002-ORFBO,1,2,1,0,2,9,0,1,42.39,0,1,3,16.0,0,1,0,1,1,0,0,1,3,1,2,65.60,593.30,0.00,0,381.51,974.81,1
1,0003-MKNFE,0,3,0,0,0,9,0,1,10.69,1,1,3,10.0,0,0,0,0,0,1,1,0,1,0,2,-4.00,542.40,38.33,10,96.21,610.28,1
5,0013-MHZWF,1,1,0,3,0,9,5,1,16.77,0,1,3,73.0,0,0,0,1,1,1,1,1,1,1,2,69.40,571.45,0.00,0,150.93,722.38,1
6,0013-SMEOE,1,4,1,0,1,71,1,1,9.96,0,1,1,14.0,1,1,1,1,1,1,1,1,2,1,1,109.70,7904.25,0.00,0,707.16,8611.41,1
7,0014-BMAQU,0,3,1,0,8,63,2,1,12.96,1,1,1,7.0,1,0,0,1,0,0,0,0,2,1,2,84.65,5377.80,0.00,20,816.48,6214.28,1


In [198]:
df_churn_model_stayed.shape

(5174, 32)

## Analise das colunas

### `Gender`

In [199]:
coluna = 'Gender'

In [200]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Gender
1    0.502408
0    0.497592
Name: proportion, dtype: float64

In [201]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Gender
0    0.507344
1    0.492656
Name: proportion, dtype: float64

### `Age`

In [202]:
coluna = 'Age'

In [203]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Age
4    0.303906
3    0.266988
2    0.266453
1    0.116640
5    0.046014
Name: proportion, dtype: float64

In [204]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Age
3    0.300928
2    0.300155
4    0.218593
1    0.157325
5    0.023000
Name: proportion, dtype: float64

### `Married`

In [205]:
coluna = 'Married'

In [206]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Married
0    0.642055
1    0.357945
Name: proportion, dtype: float64

In [207]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Married
1    0.528218
0    0.471782
Name: proportion, dtype: float64

### `Offer`

In [208]:
coluna = 'Offer'

In [209]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Offer
0    0.562333
5    0.227929
4    0.086142
2    0.054040
3    0.050829
1    0.018727
Name: proportion, dtype: float64

In [210]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Offer
0    0.546193
2    0.139737
1    0.093738
4    0.085234
5    0.073251
3    0.061848
Name: proportion, dtype: float64

### `Phone Service`

In [211]:
coluna = 'Phone Service'

In [212]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Phone Service
1    0.909042
0    0.090958
Name: proportion, dtype: float64

In [213]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Phone Service
1    0.901044
0    0.098956
Name: proportion, dtype: float64

### `Multiple Lines`

In [214]:
coluna = 'Multiple Lines'

In [215]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Multiple Lines
0    0.545211
1    0.454789
Name: proportion, dtype: float64

In [216]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Multiple Lines
0    0.590066
1    0.409934
Name: proportion, dtype: float64

### `Internet Service`

In [217]:
coluna = 'Internet Service'

In [218]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Internet Service
1    0.93954
0    0.06046
Name: proportion, dtype: float64

In [219]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Internet Service
1    0.726904
0    0.273096
Name: proportion, dtype: float64

### `Internet Type`

In [220]:
coluna = 'Internet Type'

In [221]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Internet Type
1    0.661316
2    0.164259
3    0.113965
0    0.060460
Name: proportion, dtype: float64

In [222]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Internet Type
1    0.347700
0    0.273096
2    0.259954
3    0.119250
Name: proportion, dtype: float64

### `Online Security`

In [223]:
coluna = 'Online Security'

In [224]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Online Security
0    0.842162
1    0.157838
Name: proportion, dtype: float64

In [225]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Online Security
0    0.666796
1    0.333204
Name: proportion, dtype: float64

### `Online Backup`

In [226]:
coluna = 'Online Backup'

In [227]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Online Backup
0    0.720171
1    0.279829
Name: proportion, dtype: float64

In [228]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Online Backup
0    0.63162
1    0.36838
Name: proportion, dtype: float64

### `Device Protection Plan`

In [ ]:
coluna = 'Device Protection Plan'

In [230]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Device Protection Plan
0    0.7084
1    0.2916
Name: proportion, dtype: float64

In [231]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Device Protection Plan
0    0.637225
1    0.362775
Name: proportion, dtype: float64

### `Premium Tech Support`

In [232]:
coluna = 'Premium Tech Support'

In [233]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Premium Tech Support
0    0.834136
1    0.165864
Name: proportion, dtype: float64

In [234]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Premium Tech Support
0    0.664863
1    0.335137
Name: proportion, dtype: float64

### `Streaming TV`

In [235]:
coluna = 'Streaming TV'

In [236]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Streaming TV
0    0.564473
1    0.435527
Name: proportion, dtype: float64

In [237]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Streaming TV
0    0.634132
1    0.365868
Name: proportion, dtype: float64

### `Streaming Movies`

In [238]:
coluna = 'Streaming Movies'

In [239]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Streaming Movies
0    0.562333
1    0.437667
Name: proportion, dtype: float64

In [240]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Streaming Movies
0    0.630073
1    0.369927
Name: proportion, dtype: float64

### `Streaming Music`

In [241]:
coluna = 'Streaming Music'

In [242]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Streaming Music
0    0.610487
1    0.389513
Name: proportion, dtype: float64

In [243]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Streaming Music
0    0.659838
1    0.340162
Name: proportion, dtype: float64

### `Unlimited Data`

In [244]:
coluna = 'Unlimited Data'

In [245]:
df_churn_model_churned[coluna].value_counts(normalize=True)

Unlimited Data
1    0.803638
0    0.196362
Name: proportion, dtype: float64

In [246]:
df_churn_model_stayed[coluna].value_counts(normalize=True)

Unlimited Data
1    0.626788
0    0.373212
Name: proportion, dtype: float64

## Dropando Colunas

In [247]:
drop_columns = ['Streaming Music', 'Streaming Movies', 'Streaming TV', 'Device Protection Plan', 
                'Online Backup', 'Multiple Lines', 'Gender']

In [248]:
df_churn_model = df_churn_model.drop(columns=drop_columns)

In [249]:
df_churn_model

,Customer ID,Age,Married,Number of Dependents,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Premium Tech Support,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status
0,0002-ORFBO,2,1,0,2,9,0,1,42.390000,1,3,16.0,0,1,1,3,1,2,65.60,593.30,0.00,0,381.51,974.81,1
1,0003-MKNFE,3,0,0,0,9,0,1,10.690000,1,3,10.0,0,0,0,1,0,2,-4.00,542.40,38.33,10,96.21,610.28,1
2,0004-TLHLJ,3,0,0,0,4,5,1,33.650000,1,1,30.0,0,0,1,1,1,1,73.90,280.85,0.00,0,134.60,415.45,0
3,0011-IGKFF,5,1,0,1,13,4,1,27.820000,1,1,4.0,0,0,1,1,1,1,98.00,1237.85,0.00,0,361.66,1599.51,0
4,0013-EXCHZ,4,1,0,3,3,0,1,7.380000,1,1,11.0,0,1,1,1,1,2,83.90,267.40,0.00,0,22.14,289.54,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,1,0,0,0,13,4,1,46.680000,1,2,59.0,1,1,1,3,0,2,55.15,742.90,0.00,0,606.84,1349.74,1
7039,9992-RRAMN,2,1,0,1,22,4,1,16.200000,1,1,17.0,0,0,1,1,1,1,85.10,1873.70,0.00,0,356.40,2230.10,0
7040,9992-UJOEL,1,0,0,0,2,5,1,18.620000,1,2,51.0,0,0,1,1,1,2,50.30,92.75,0.00,0,37.24,129.99,2
7041,9993-LHIEB,1,1,0,5,67,1,1,2.120000,1,3,58.0,1,1,1,2,0,2,67.85,4627.65,0.00,0,142.04,4769.69,1


In [250]:
filtered_path = os.path.join(PATH, 'churn_model_filtered.csv')
df_churn_model.to_csv(filtered_path, index=False)